# Два указателя
## Город Че
Пусть у нас есть следующая задача:

"В центре города Че есть пешеходная улица - одно из самых популярных мест для прогулок жителей города. По этой улице очень приятно гулять, ведь вдоль улицы расположено $n$ забавных памятников.

Девочке Маше из города Че нравятся два мальчика из ее школы, и она никак не может сделать выбор между ними. Чтобы принять окончательное решение, она решила назначить обоим мальчикам свидание в одно и то же время. Маша хочет выбрать два памятника на пешеходной улице, около которых мальчики будут ее ждать. При этом она хочет выбрать такие памятники, чтобы мальчики не увидели друг друга. Маша знает, что из-за тумана мальчики увидят друг друга только в том случае, если они будут на расстоянии не более $r$ метров.

Маше заинтересовалась, а сколько способов есть выбрать два различных памятника для организации свиданий."

То есть нам нужно найти количество элементов $a$ и $b$ в массиве, такие что $|a - b| > r$. Такую задачу мы уже давно умеем решать с помощью бинарного поиска. Будем считать, что массив уже отсортирован. Для каждого элемента $a$ найдем первый справа элемент $b$, который входит в ответ в паре с $a$. Нетрудно заметить, что все элементы большие $b$ также входят в ответ. Итоговая асимптотика $O(n\log n)$.

А можно ли быстрее?

Действительно, заметим, что ответ для конкретного элемента - это суффикс отсортированного массива. Тогда для элементов $a$ и $b$ $(b > a)$ количество подходящий элементов для $b$ будет не больше, чем для $a$. Давайте тогда перебирать элементы в отсортированном массиве и поддерживать для каждого $a$ первый элемент, который входит в ответ в паре с $a$.

In [0]:
int a[n], r;

cin >> r;

for (int i = 0; i < n; ++i) {
    cin >> a[i];
}

int first = 0, ans = 0;
for (int i = 0; i < n; ++i) {
    while (first != n && a[first] - a[i] <= r) {
        first++;
    }
    
    ans += n - first;
}

cout << ans << endl;

За сколько же работает это решение? С виду может показаться, что за $O(n^2)$, но давайте посмотрим сколько раз меняется значение переменной $first$. Так как оно изначально равняется нулю, только увеличивается и не может превысить $n$, то суммарно операций мы сделаем $O(n)$. Только что мы увидели метод двух указателей, где в роли первого указателя выступал $i$, а в роли второго $j$.

Так когда же можно использовать метод двух указателей? Посмотрим чем мы пользовались. Нам было важно, что исходный массив и наша функция ответа - монотонные функции. Собственно только что мы сформулировали критерий для того, чтобы пользоваться этим методом.

Давайте разберем еще примеры.

## Слияние
Пусть у нас есть два отсортированных по неубыванию массива размера $n$ и $m$. Хотим получить отсортированный массив размера $n + m$ из исходных. Воспользуемся методом двух указателей. Первый указатель будет указывать на начало первого массива, а второй, соответственно, на начало второго. Из двух текущих элементов, на которые указывают указатели, выберем наименьший и положим на соответствующую позицию в новом массиве, после чего сдвинем указатель. Продолжим этот процесс пока в обоих массивах не закончатся элементы. Тогда код будет выглядеть следующим образом:

In [0]:
int a[n + 1], b[m + 1], res[n + m];

a[n] = INF; // Создаем в конце массива фиктивный элемент, который заведомо больше остальных
b[m] = INF; // Чтобы избежать лишних случаев

for (int i = 0; i < n; ++i) {
    cin >> a[i];
}

for (int j = 0; j < m; ++j) {
    cin >> a[j];
}

int i = 0, j = 0;
for (int k = 0; k < n + m; ++k) {
    if (a[i] < b[j]) {
        res[k] = a[i];
        i++;
    } else {
        res[k] = b[j];
        j++;
    }
}

Итоговая ассимптотика: $O(n + m)$.

# Сортировка слияением

Сейчас мы научимся для нас первому алгоритму, который сортирует элементы за $O(n\log n)$.

Пусть у нас есть какой-то массив.

In [0]:
int a[8] = {7, 2, 5, 6, 1, 3, 4, 8};

Сделаем такое предположение. Пусть мы уже умеем как-то сортировать массив размера $n$. Тогда научимся сортировать массив размера $2n$.
Давайте разобьем наш массив на две половины, отсортируем каждую из них, а после это сделаем слияние двух массивов, которое мы научились делать за $O(n)$ в данных условиях. Также заметим, что массив размера $1$ уже отсортирован, тогда мы можем делать это процедуру рекурсивно. Тогда для данного массива $a$ это будет выглядеть следующим образом:

In [0]:
// (7 2 5 6 1 3 4 8)
// (7 2 5 6) (1 3 4 8)
// (7 2) (5 6) (1 3) (4 8)
// (2 7) (5 6) (1 3) (4 8)
// (2 5 6 7) (1 3 4 8)
// (1 2 3 4 5 6 7 8)

#include <algorithm> // Воспользуемся встроенной функцией merge

void merge_sort(vector<int> &v, int l, int r) { // v - вектор, который хотим отсортировать
    if (r - l == 1) {                            // l и r - полуинтервал, который хотим отсортировать
        return;
    }
    
    int mid = (l + r) / 2;
    merge_sort(v, l, mid);
    merge_sort(v, mid, r);
    vector<int> temp(r - l); // временный вектор
    merge(v.begin() + l, v.begin() + mid, v.begin() + mid, v.begin() + r, c.begin());
    for (int i = 0; i < r - l; ++i) {
        v[i + l] = temp[i];
    }
    return;
}

Так сколько же работает это решение?

Пускай $T(n)$ — время сортировки массива длины $n$, тогда для сортировки слиянием справедливо $T(n)=2T(n/2)+O(n)$ 
$O(n)$ — время, необходимое на то, чтобы слить два массива длины n. Распишем это соотношение:

$T(n)=2T(n/2)+O(n)=4T(n/4)+2O(n)=\ldots=T(1)+\log(n)O(n)=O(n\log(n)).$

В итоге мы получили самую быструю ассимптотически сортировку!

# Инверсии

Пусть у нас есть некоторая перестановка $a$. Инверсией называется пара индексов $i$ и $j$ такая, что $i < j$ и $a[i] > a[j]$. Поставим задачу найти количество инверсий в данной перестановке. Очевидно, что эта задача лего решается обычным перебором двух индексов за $O(n^2)$:

In [0]:
int a[n], ans = 0;

for (int i = 0; i < n; ++i) {
    for (int j = i + 1; j < n; ++j) {
        if (a[i] > a[j]) {
            ans++;
        }
    }
}

cout << ans << endl;

Внезапно эту задачу можно решить используя сортировку слиянием, слегка модифицируя её. Оставим ту же идею. Пусть мы умеем находить количество инверсий в массиве размера $n$, научимся находить количество инверсий в массиве размера $2n$.

В случае массива длины $1$ количество инверсий $0$. Значит, достаточно придумать лишь как считать ответ для всего массива, зная ответ для его половин. Причем так как мы модифицируем сортировку слиянием, то после вычисления ответа для меньших частей мы получим два отсортированных массива. Эту задачу можно решить методом двух указателей. Написание итогового алгоритма оставим в качестве упражнения.

## Задание
Решите как можно больше задач из контеста ...